# Data cleaning and aggregation

In this script we will clean the given datasets, and import them to the local SQLite database.

| atribute           | info                                                                                                                         | source | attributes| year |
| ------------------ | ---------------------------------------------------------------------------------------------------------------------------- | ------ | ---- | ---- |
| year | x | teams | attributes  | current |
| tmID | x | teams | attributes  | current |
| playoff | x | teams | attributes  | current |
| averageWinRate | If previous year is not available, use the yearly average for the selected attributes. | teams_players | won, lost  | previous |
| averagePoints | If previous year is not available, use the yearly average for the selected attributes. | teams_players | points  | previous |
| averageRebounds | If previous year is not available, use the yearly average for the selected attributes. | teams_players | rebounds  | previous |
| averageAssists | If previous year is not available, use the yearly average for the selected attributes. | teams_players | assists  | previous |
| averageSteals | If previous year is not available, use the yearly average for the selected attributes. | teams_players | steals  | previous |
| averageBlocks | If previous year is not available, use the yearly average for the selected attributes. | teams_players | blocks  | previous |
| averageTurnovers | If previous year is not available, use the yearly average for the selected attributes. | teams_players | turnovers  | previous |
| averageFGRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | fgAttempted, fgMade  | previous |
| averageFTRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | ftAttempted, ftMade  | previous |
| averageThreeRatio | If previous year is not available, use the yearly average for the selected attributes. | teams_players | threeAttempted, threeMade  | previous |
| coachWinRate | If previous year is not available, use the yearly average for the selected attributes. | coaches | won, lost  | previous |
| numberOfAwardedPlayers | If previous year is not available, use the yearly average for the selected attributes. | awards_players | *count  | previous |

SQL Not used

In [1]:
# Create connection layer

import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.row_factory = sqlite3.Row
    except Error as e:
        print(e)

    return conn
        
def delete_all_data(conn):
    cur = conn.cursor()
    print("Deleted ", cur.execute("DELETE FROM coaches").rowcount, " from coaches table.")
    print("Deleted ", cur.execute("DELETE FROM awards_players").rowcount, " from awards_players table.")
    print("Deleted ", cur.execute("DELETE FROM players_teams").rowcount, " from players_teams table.")
    print("Deleted ", cur.execute("DELETE FROM series_post").rowcount, " from series_post table.")
    print("Deleted ", cur.execute("DELETE FROM teams_post").rowcount, " from teams_post table.")
    print("Deleted ", cur.execute("DELETE FROM teams").rowcount, " from teams table.")
    print("Deleted ", cur.execute("DELETE FROM players").rowcount, " from players table.")


database = "./datasets/original/sqlite_database_g79.db"

# create a database connection
conn = create_connection(database)

In [2]:
# Query test
def get_teams(conn):
  cur = conn.cursor()
  cur.execute("SELECT * FROM teams LIMIT 10")
  
  return cur.fetchall()

# --------------------------------------------------------------

allTeams = get_teams(conn)

for i in allTeams:
  print(i["name"])

Atlanta Dream
Atlanta Dream
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Charlotte Sting
Chicago Sky


In [10]:
delete_all_data(conn)

Deleted  162  from coaches table.
Deleted  95  from awards_players table.
Deleted  1876  from players_teams table.
Deleted  70  from series_post table.
Deleted  80  from teams_post table.
Deleted  142  from teams table.
Deleted  893  from players table.


In [3]:
# Close connection 

conn.close()

## Cleaning and aggregation

Initialize all csv paths and selected attributes.

In [1]:
players_teams_csv_file_path = "./datasets/original/players_teams.csv"
coaches_csv_file_path = "./datasets/original/coaches.csv"
teams_csv_file_path = "./datasets/original/teams.csv"
awards_players_csv_file_path = "./datasets/original/awards_players.csv"

aggregated_table_csv_file_path = "./datasets/generated/aggregated_table.csv"

players_teams_selected_attributes = ["playerID", "year", "tmID", "points", "rebounds", "assists", "steals", "blocks", "turnovers", "fgAttempted", "fgMade", "ftAttempted", "ftMade", "threeAttempted", "threeMade"]
coaches_selected_attributes = ["coachID", "year", "tmID", "stint", "won", "lost"]
teams_selected_attributes = ["year", "tmID", "won", "lost", "playoff"]
awards_players_selected_attributes = ["playerID", "year"]

aggregated_attributes = ["tmID", "year", "playoff", "averageWinRate", "averagePoints", "averageRebounds", "averageAssists", "averageSteals", "averageBlocks", "averageTurnovers", "averageFGRatio", "averageFTRatio", "averageThreeRatio", "coachWinRate", "numberOfAwardedPlayers", "listOfPlayers"]


Load all data and select only wanted attributes from DataFrames.

In [2]:
import pandas as pd

initial_df_teams_players = pd.read_csv(players_teams_csv_file_path)
df_teams_players = initial_df_teams_players[players_teams_selected_attributes]

initial_df_coaches = pd.read_csv(coaches_csv_file_path)
df_coaches = initial_df_coaches[coaches_selected_attributes]

initial_df_teams = pd.read_csv(teams_csv_file_path)
df_teams = initial_df_teams[teams_selected_attributes]

initial_df_awards_players = pd.read_csv(awards_players_csv_file_path)
df_awards_players = initial_df_awards_players[awards_players_selected_attributes]


In [3]:
import pandas as pd

df_final_temp = df_teams[["tmID", "year", "playoff"]]
df_final_temp = df_final_temp.sort_values(["tmID", "year"], ascending=False)

df_final_temp["players"] = "" 
df_final_temp = df_final_temp.reset_index()
df_final_temp = df_final_temp.drop("index", axis=1)
df_final_temp.head()

for index, row in df_final_temp.iterrows():
  currentYear = row["year"]
  currentTeamId = row["tmID"]
  
  mask = df_teams_players[(df_teams_players["year"] == currentYear ) & (df_teams_players["tmID"] == currentTeamId)]
  
  df_final_temp["players"][index] = mask["playerID"].values


df_final_temp.head(20)

C:\Users\Zan\AppData\Local\Temp\ipykernel_23268\3599968181.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_temp["players"][index] = mask["playerID"].values


,tmID,year,playoff,players
0,WAS,10,Y,"[ajavoma01w, beardal01w, blueni01w, colemma01w..."
1,WAS,9,N,"[beardal01w, blueni01w, currimo01w, gardnan01w..."
2,WAS,8,N,"[beardal01w, blueni01w, currimo01w, goringi01w..."
3,WAS,7,Y,"[beardal01w, blueni01w, byearla01w, jamesta01w..."
4,WAS,6,N,"[amachma01w, beardal01w, brownki01w, choneka01..."
5,WAS,5,Y,"[beardal01w, brownki01w, choneka01w, dalesst01..."
6,WAS,4,N,"[brownki01w, burgean01w, dalesst01w, henniso01..."
7,WAS,3,Y,"[brownki01w, bullevi01w, burgean01w, dalesst01..."
8,WAS,2,N,"[aldrima01w, bauerca01w, bullevi01w, burgean01..."
9,WAS,1,Y,"[aldrima01w, anderke01w, bullevi01w, campbmi01..."


Calculate all the averages for all tables, so we can populate missing values later.

In [4]:
df_teams_players_average = df_teams_players.groupby("year").agg("mean", numeric_only=True)
df_teams_average = df_teams.groupby("year").agg("mean", numeric_only=True)
df_coaches_average = df_coaches.groupby("year").agg("mean", numeric_only=True)

df_teams_players_average = df_teams_players_average.reset_index()
df_teams_average = df_teams_average.reset_index()
df_coaches_average = df_coaches_average.reset_index()

df_teams_average.head(10)


,year,won,lost
0,1,16.0,16.0
1,2,16.0,16.0
2,3,16.0,16.0
3,4,17.0,17.0
4,5,17.0,17.0
5,6,17.0,17.0
6,7,17.0,17.0
7,8,17.0,17.0
8,9,17.0,17.0
9,10,17.0,17.0


In [5]:
# loop all temp rows
df_final = df_final_temp
new_columns = ["averageWinRate", "averagePoints", "averageRebounds", "averageAssists", "averageSteals", "averageBlocks", "averageTurnovers", "averageFGRatio", "averageFTRatio", "averageThreeRatio", "coachWinRate", "numberOfAwardedPlayers"]
df_final[new_columns] = ""

for idx in range(len(df_final)):
  row = df_final.iloc[idx]
    
  currentYear = row["year"]
  currentTeamId = row["tmID"]
  players = row["players"]
  coach_id = df_coaches[(df_coaches["tmID"] == currentTeamId) & (df_coaches["year"] == currentYear)]["coachID"].values[0]
  
  if(currentYear == 1):
    continue
  
  previousYearExists = any((df_final["year"] == currentYear - 1) & (df_final["tmID"] == currentTeamId))
  
  # for all players get data from currentYear - 1
  players_data = df_teams_players[(df_teams_players["playerID"].isin(players)) & (df_teams_players["year"] == currentYear - 1)]
  teams_data = df_teams[(df_teams["tmID"] == currentTeamId) & (df_teams["year"] == currentYear - 1)]
  coach_data = df_coaches[(df_coaches["coachID"] == coach_id) & (df_coaches["year"] == currentYear - 1)]
  awards_data = df_awards_players[(df_awards_players["playerID"].isin(players)) & (df_awards_players["year"] < currentYear)]
  
  # Populate missing teams_players data
  missing_players = []
  
  for player in list(players):
    if not player in players_data["playerID"].values:
      missing_players.append(player)
  
  last_year_players_average = df_teams_players_average[df_teams_players_average["year"] == (currentYear - 1)]
  last_year_players_average["tmID"] = currentTeamId
  
  # populate players_data with avg players data
  for player in missing_players:
    last_year_players_average["playerID"] = player
    players_data = players_data.append(last_year_players_average)
    
  # Populate missing coaches data
  if len(coach_data) == 0:
    last_year_coaches_average = df_coaches_average[df_coaches_average["year"] == (currentYear - 1)]
    last_year_coaches_average["tmID"] = currentTeamId
    last_year_coaches_average["coachID"] = coach_id
    
    coach_data = coach_data.append(last_year_coaches_average)
  
  # Populate missing teams data
  if len(teams_data) == 0:
    last_year_teams_average = df_teams_average[df_teams_average["year"] == (currentYear - 1)]
    last_year_teams_average["tmID"] = currentTeamId
    
    teams_data = teams_data.append(last_year_teams_average)
  
  # calculate attributes
  averagePoints = players_data["points"].mean()
  averageRebounds = players_data["rebounds"].mean()
  averageAssists = players_data["assists"].mean()
  averageSteals = players_data["steals"].mean()
  averageBlocks = players_data["blocks"].mean()
  averageTurnovers = players_data["turnovers"].mean()
  averageFGRatio = (players_data["fgMade"] / players_data["fgAttempted"]).mean()
  averageFTRatio = (players_data["ftMade"] / players_data["ftAttempted"]).mean()
  averageThreeRatio = (players_data["threeMade"] / players_data["threeAttempted"]).mean()

  averageWinRate = (teams_data["won"] / (teams_data["won"] + teams_data["lost"])).mean()
  coachWinRate = (coach_data["won"] / (coach_data["won"] + coach_data["lost"])).mean()
  numberOfAwardedPlayers = len(awards_data)
  
  df_final["averagePoints"][idx] = averagePoints
  df_final["averageRebounds"][idx] = averageRebounds
  df_final["averageAssists"][idx] = averageAssists
  df_final["averageSteals"][idx] = averageSteals
  df_final["averageBlocks"][idx] = averageBlocks
  df_final["averageTurnovers"][idx] = averageTurnovers
  df_final["averageFGRatio"][idx] = averageFGRatio
  df_final["averageFTRatio"][idx] = averageFTRatio
  df_final["averageThreeRatio"][idx] = averageThreeRatio
  df_final["averageWinRate"][idx] = averageWinRate
  df_final["coachWinRate"][idx] = coachWinRate
  df_final["numberOfAwardedPlayers"][idx] = numberOfAwardedPlayers
  
df_final = df_final[df_final["year"] > 1]
df_final = df_final.drop("players", axis=1)

# write to JSON
df_json = df_final.to_json(orient="records")

new_file = open("./datasets/generated/cleaned_aggregated_data.json", "w")
new_file.writelines(df_json)
new_file.close()

df_final.head(10)


C:\Users\Zan\AppData\Local\Temp\ipykernel_23268\4106948462.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year_players_average["tmID"] = currentTeamId
C:\Users\Zan\AppData\Local\Temp\ipykernel_23268\4106948462.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year_players_average["playerID"] = player
C:\Users\Zan\AppData\Local\Temp\ipykernel_23268\4106948462.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,10,Y,0.294118,189.512143,84.572143,42.982857,19.696786,7.073929,44.290357,0.408655,0.717564,0.306577,0.5,0
1,WAS,9,N,0.470588,189.708943,82.593336,36.207412,20.147909,8.820809,38.090105,0.402735,0.800336,0.276818,0.5,2
2,WAS,8,N,0.529412,239.624444,95.275556,50.680741,25.66,10.042222,44.559259,0.434105,0.760861,0.303131,0.529412,2
3,WAS,7,Y,0.470588,216.456929,88.321161,42.381086,23.959738,8.20412,39.441011,0.422553,0.743379,0.337256,0.470588,2
4,WAS,6,N,0.5,193.30631,92.482727,39.100875,21.625518,11.273146,39.394288,0.429846,0.706172,0.334885,0.4375,1
5,WAS,5,Y,0.264706,240.308333,107.468519,54.680556,23.091667,9.691667,44.64537,0.409827,0.702331,0.30017,0.5,1
6,WAS,4,N,0.53125,167.989247,78.873425,43.032873,17.364055,6.389555,35.870968,0.401379,0.727574,0.310486,0.53125,1
7,WAS,3,Y,0.3125,157.101648,91.43956,37.078297,21.982143,11.538462,39.072802,0.387412,0.690721,0.276616,0.5,0
8,WAS,2,N,0.4375,216.313874,92.934066,39.79739,23.455357,11.973214,44.57967,0.439097,0.696891,0.319172,0.5,0
10,UTA,3,Y,0.59375,230.494083,109.550296,50.776627,21.596154,14.887574,49.616864,0.413616,0.749187,0.255318,0.736842,0
